In [1]:
from simulation import citygraph_dataset
from learning import inductive_route_learning
from omegaconf import OmegaConf, DictConfig

In [3]:
import sys
sys.path.append('/Users/sashamorozov/Documents/Code/GNN/TNDP_learning')

In [2]:
# from simulation.city_graphdataset import DynamicCityGraphDataset, MST, OUT_KNN, IN_KNN, VORONOI, GRID4, GRID8, CIRC, SMALLWORLD, MIXED
from pathlib import Path
import shutil
from tqdm import tqdm

In [ ]:
dataset = citygraph_dataset.DynamicCityGraphDataset(
    min_nodes=50,
    max_nodes=50,
    edge_keep_prob=0.7,
    data_type=citygraph_dataset.MIXED,  # or any other type you want
    directed=False,
    fully_connected_demand=True,  # default SIDE_LENGTH_M
    mumford_style=True,
    pos_only=False
)

# Generate graphs
n_graphs = 100  # number of graphs you want to generate
graphs = [dataset.generate_graph(draw=True) for _ in tqdm(range(n_graphs))]

# Save the dataset
save_path = Path('./output_graphs')  # adjust path as needed
if not save_path.exists():
    save_path.mkdir(parents=True)

import pickle
with open(save_path / 'raw_graphs.pkl', 'wb') as ff:
    pickle.dump(graphs, ff)

In [5]:
import pickle

with open('/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/output_graphs/mixed/raw_graphs.pkl', 'rb') as f:
    graphs = pickle.load(f)

In [ ]:
graphs

In [32]:
cfg = OmegaConf.load("cfg/ppo_20nodes.yaml")  # путь к вашему конфигу
OmegaConf.resolve(cfg)

In [39]:
OmegaConf.resolve(cfg)

In [ ]:
base_cfg = OmegaConf.load("cfg/ppo_20nodes.yaml")
exp_cfg = OmegaConf.load("cfg/experiment/standard.yaml")
cost_cfg = OmegaConf.load("cfg/experiment/cost_function/multi.yaml")
sim_cfg = OmegaConf.load("cfg/experiment/cost_function/multi.yaml")

# Ручное слияние
cfg = OmegaConf.merge(base_cfg, {"experiment": exp_cfg, "cost_function": cost_cfg})

In [14]:
from omegaconf import OmegaConf
from hydra import initialize_config_dir, compose

with initialize_config_dir(config_dir="/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/cfg", job_name="app"):
    cfg = compose(config_name="ppo_20nodes_copy.yaml")  # загрузит config.yaml + всё из defaults
    # cfg = OmegaConf.to_container(cfg, resolve=False)  # опционально — привести к dict
    print(cfg)

{'ppo': {'n_iterations': 200, 'val_period': 10, 'n_epochs': 1, 'minibatch_size': 25, 'horizon': 120, 'epsilon': 0.2, 'use_gae': True, 'gae_lambda': 0.95}, 'discount_rate': 0.95, 'diff_reward': True, 'baseline_lr': 0.0005, 'entropy_weight': 0.0, 'batch_size': 25, 'reward_scale': 1.0, 'lr': 0.0016134816080499328, 'decay': 0.0008404361781997002, 'optimizer': 'Adam', 'eval': {'n_routes': 10, 'min_route_len': 2, 'max_route_len': 12}, 'dataset': {'type': 'pickle', 'kwargs': {'path': 'output_graphs/mixed', 'space_scale': 0.6, 'demand_scale': 0.2}}, 'experiment': {'logdir': 'training_logs', 'anomaly': False, 'cpu': False, 'seed': 0, 'symmetric_routes': True, 'cost_function': {'type': 'mine', 'kwargs': {'mean_stop_time_s': 0, 'avg_transfer_wait_time_s': 300, 'demand_time_weight': 0.5, 'route_time_weight': 0.5, 'constraint_violation_weight': 5.0, 'variable_weights': True, 'pp_fraction': 0.33, 'op_fraction': 0.33}}}, 'model': {'common': {'dropout': 0.0, 'nonlin_type': 'ReLU', 'embed_dim': 64}, 'r

/var/folders/zx/byb3bcds50s1lwbdr2mwq18r0000gn/T/ipykernel_5405/1973714916.py:4: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize_config_dir(config_dir="/Users/sashamorozov/Documents/Code/GNN/TNDP_learning/cfg", job_name="app"):


In [15]:
cfg

{'ppo': {'n_iterations': 200, 'val_period': 10, 'n_epochs': 1, 'minibatch_size': 25, 'horizon': 120, 'epsilon': 0.2, 'use_gae': True, 'gae_lambda': 0.95}, 'discount_rate': 0.95, 'diff_reward': True, 'baseline_lr': 0.0005, 'entropy_weight': 0.0, 'batch_size': 25, 'reward_scale': 1.0, 'lr': 0.0016134816080499328, 'decay': 0.0008404361781997002, 'optimizer': 'Adam', 'eval': {'n_routes': 10, 'min_route_len': 2, 'max_route_len': 12}, 'dataset': {'type': 'pickle', 'kwargs': {'path': 'output_graphs/mixed', 'space_scale': 0.6, 'demand_scale': 0.2}}, 'experiment': {'logdir': 'training_logs', 'anomaly': False, 'cpu': False, 'seed': 0, 'symmetric_routes': True, 'cost_function': {'type': 'mine', 'kwargs': {'mean_stop_time_s': 0, 'avg_transfer_wait_time_s': 300, 'demand_time_weight': 0.5, 'route_time_weight': 0.5, 'constraint_violation_weight': 5.0, 'variable_weights': True, 'pp_fraction': 0.33, 'op_fraction': 0.33}}}, 'model': {'common': {'dropout': 0.0, 'nonlin_type': 'ReLU', 'embed_dim': 64}, 'r

In [16]:
inductive_route_learning.setup_and_train(cfg)



  0%|          | 0/200 [05:10<?, ?it/s]


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 62 but got size 100 for tensor number 1 in the list.